In [ ]:
import os
print(os.getcwd())
import pandas as pd
import sys
import numpy as np
import datetime as dt
import os
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append("../../analyzer/getData")
import configparser
config = configparser.ConfigParser()
config.read('../../config.properties')
directorio=config.get('ARCHIVOS', 'archivos_h5s_precios')
print(directorio)
import bdStocks
bd= bdStocks.getData()
sectores=None
fechaI="2008-12-01"
fechaF="2022-06-01"
periodoIndice="M"
fechaI=dt.datetime.strptime(fechaI, '%Y-%m-%d')
fechaF=dt.datetime.strptime(fechaF, '%Y-%m-%d')
files=os.listdir(directorio)
h5s= pd.HDFStore(directorio+files[0], "r")
precios=h5s["data"]
h5s.close()
ex=files[0].split("_")[-1].split(".")[0]
precios["exchange"]=ex
exchanges=[ex]
time_range = pd.date_range(fechaI, fechaF,freq=periodoIndice,normalize=True)
time_range=[e.replace(day=1) for e in time_range]
for file in files[1:]:
    h5s=pd.HDFStore(directorio+file, "r")
    dat_aux=h5s["data"]
    ex=file.split("_")[-1].split(".")[0]
    dat_aux["exchange"]=ex
    precios=pd.concat([precios,dat_aux])
    exchanges.append(ex)
precios.set_index("exchange",inplace=True,append=True)
sectores=bd.getSectors()

In [ ]:
exchange="US"
indice="nasdaq100"
datos=bd.executeQueryDataFrame("select  stock,fecha,adjusted_close*commonStockSharesOutstanding as market_cap from\
 ratios_results inner join indices on indices.company=ratios_results.stock where indices.indice=%s\
 and ratios_results.exchange=%s order by fecha asc;",(indice,exchange))

In [ ]:
precios=bd.executeQueryDataFrame("select  stock,fecha,adjusted_close from\
 {}_precios inner join indices on indices.company={}_precios.stock where indices.indice=%s\
 order by fecha asc;".format(exchange,exchange),(indice,))

In [ ]:
precios.tail()

In [ ]:
precios["fecha"]=pd.to_datetime(precios.fecha)
precios=precios.set_index(["fecha","stock"],drop=False)
precios1=precios.loc[precios.fecha>dt.datetime(2021,10,1)]
precios2=precios.loc[precios.fecha>dt.datetime(2022,1,1)]
diferencias={
}
diferenciasAno={}
for e in np.unique(precios1.index.get_level_values(1)):
    diferencias[e]=((precios1.loc[:,e,:].adjusted_close[-1]-max(precios1.loc[:,e,:].adjusted_close))/max(precios1.loc[:,e,:].adjusted_close))
    diferenciasAno[e]=(precios2.loc[:,e,:].adjusted_close[-1]-precios2.loc[:,e,:].adjusted_close[0])/precios2.loc[:,e,:].adjusted_close[0]

In [ ]:
sectors=bd.getSectors("US")
names=bd.executeQuery("select code,name from stocks where exchange='nasdaq'")
names={e[0]:e[1] for e in names}
descripciones=bd.executeQuery("select stock,description from descriptions where exchange='US'")
descripciones={e[0]:e[1] for e in descripciones}

In [ ]:
datos=datos.groupby("stock",as_index=False).last()
datos=datos.sort_values(by="market_cap")
datos["sector"]=datos.stock.transform(lambda x: sectors[x] if x in sectors.keys() else None)
datos["diff"]=datos.stock.transform(lambda x: diferencias[x] if x in diferencias.keys() else None)
datos["diffAno"]=datos.stock.transform(lambda x: diferenciasAno[x] if x in diferencias.keys() else None)
datos["name"]=datos.stock.transform(lambda x: names[x] if x in names.keys() else None)

In [ ]:
datos=datos.sort_values(by="diffAno")
datos.head(50)


In [ ]:
indice=bd.executeQueryDataFrame("select * from preciosindices where indice='NASDAq' ORDER BY date  ASC",None)
indice["Date"]=pd.to_datetime(indice.Date)
indice.set_index("Date",inplace=True)

In [ ]:
datosRes=bd.executeQueryDataFrame("select * from ratios_results where report_date is not null order by report_date asc",None)

In [ ]:
datosRes.set_index(["fecha","stock","exchange"],inplace=True)

In [ ]:
stocks=iter(list(datos.stock))

In [ ]:
stock=next(stocks)
names[stock],descripciones[stock],stock

In [ ]:

#pre=precios.loc[precios.fecha>dt.datetime(2015,1,1)].loc[:,stock,:].droplevel("stock")
pre=precios.loc[:,stock,:].droplevel("stock")
pre["indice"]=indice["Close"]
pre=pre.loc[pre.index.get_level_values(0)>dt.datetime(2015,1,1)]
plt.plot(pre.index.get_level_values(0),(pre.indice-pre.indice[0])/pre.indice[0])
plt.plot(pre.index.get_level_values(0),(pre.adjusted_close-pre.adjusted_close[0])/pre.adjusted_close[0])
plt.xticks(rotation = 'vertical')
u=pre.loc[:,["indice","adjusted_close"]]
u["fecha"]=u.index
#u=u.groupby([u.fecha.dt.year,u.fecha.dt.month]).head(1)
corr=u.corr()
corr


In [ ]:

f_aux=datosRes.loc[:,stock,"US",:].reset_index()
f_aux.loc[len(f_aux)-1,"adjusted_close"]=precios.loc[:,stock,:].adjusted_close[-1]
f_aux["fecha"]=pd.to_datetime(f_aux.fecha)
f_aux=f_aux.loc[f_aux.fecha>dt.datetime(2012,1,1)]
f_aux["per"]=(f_aux.adjusted_close*f_aux.commonStockSharesOutstanding)/f_aux.netIncome
print(list(f_aux.per)[-1]
      /4,list(f_aux.earnings_per)[-1]/4)
c=[]
for columna in f_aux.columns:
    if f_aux[columna].dtypes==float:
        if f_aux.isna().sum()[columna]<f_aux.shape[0]/2:
            c.append(columna)
figure,ax=plt.subplots(int(len(c)/2)+1,2,figsize=(20,40))
i=j=0
for u in c:
    ax[i,j].plot(f_aux["fecha"],f_aux[u],marker="o")
    ax[i,j].title.set_text(stock+"_"+u)
    j+=1
    if j==2:
        j=0
        i+=1


In [ ]:
f_aux.loc[:,["netIncome","adjusted_close"]].corr()

Acciones seleccionadas: AMD, NVDA, AMAT, SWKS, ADSK, QCOM, ADBE, ZM, NFLX, BIDU, PDD, PYPL, FB, DOCU.
Criterio de seleccion: beneficios crecientes los ultimos meses y caidas de precio superior a 30% (caida debido a caida de mercado de 2022-01-01 a 2022-05-01) o caidas yoy superiores a 60% y valoracions inferiores a 2019 para acciones consolidadas.

In [ ]:
columnas=["AMD",'NVDA',"AMAT", 'SWKS',"ADSK","QCOM","ADBE","ZM","NFLX","BIDU","PDD","PYPL","FB","DOCU"]
semi=precios.loc[precios.index.get_level_values(1).isin(columnas)]
semi=semi.loc[semi.fecha>dt.datetime(2015,1,1)]
prec=pd.DataFrame()
prec.index=semi.droplevel(1).index
for e in columnas:
    prec[e]=semi.loc[semi.stock==e].droplevel(1).adjusted_close

In [ ]:

df_numerico=prec
fig, ax = plt.subplots(figsize = (15,15))
ax.pcolormesh(np.arange(len(df_numerico.columns)+1), np.arange(len(df_numerico.columns)+1), df_numerico.corr().values, cmap = 'Blues')
plt.xticks(np.arange(len(df_numerico.columns)),df_numerico.columns, rotation = 'vertical');
plt.yticks(np.arange(len(df_numerico.columns)), df_numerico.columns, rotation = 'horizontal');
round(prec.corr(),3)

In [ ]:

c=[]
for columna in ["netIncome","per","earnings","earnings_per","totalRevenue","ebitda","adjusted_close"]:
    if f_aux[columna].dtypes==float:
        if f_aux.isna().sum()[columna]<f_aux.shape[0]/2:
            c.append(columna)

for u in c:
    
    plt.figure(figsize=(15,15))
    for stock in columnas:
            
                f_aux=datosRes.loc[:,stock,"US",:].reset_index()
                f_aux.loc[len(f_aux)-1,"adjusted_close"]=precios.loc[:,stock,:].adjusted_close[-1]
                f_aux["fecha"]=pd.to_datetime(f_aux.fecha)
                f_aux=f_aux.loc[f_aux.fecha>dt.datetime(2012,1,1)]
                f_aux["per"]=(f_aux.adjusted_close*f_aux.commonStockSharesOutstanding)/f_aux.netIncome
                plt.plot(f_aux["fecha"],(f_aux[u]-f_aux[u].mean())/f_aux[u].std(),marker="o")
                plt.title(u)
    plt.legend(columnas)
        

In [ ]:
stock="ANSS"
dat=bd.executeQueryDataFrame("select * from us_FUNDAMENTAL WHERE STOCK=%s",(stock,))
dat2=precios.loc[:,stock,:,:].loc[precios.fecha>dt.datetime(2016,1,1)]
f_aux=dat
f_aux["fecha"]=pd.to_datetime(f_aux.fecha)
f_aux=f_aux.loc[f_aux.fecha>dt.datetime(2012,1,1)]
c=[]


In [ ]:
f_aux.tail(10).loc[:,["fecha","netIncome"]]

In [ ]:
c=[]
for columna in ["netIncome","ebitda","totalRevenue"]:
    if f_aux[columna].dtypes==float:
            if f_aux.isna().sum()[columna]<f_aux.shape[0]/2:
                c.append(columna)
figure,ax=plt.subplots(int(len(c)/2)+1,2,figsize=(20,20))
i=j=0
for u in c:
    ax[i,j].plot(f_aux["fecha"],f_aux[u],marker="o")
    ax[i,j].title.set_text(stock+"_"+u)
    j+=1
    if j==2:
        j=0
        i+=1
ax[i,j].plot(dat2["fecha"],dat2["adjusted_close"])
ax[i,j].title.set_text(stock+"_Adjusted close")